![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use AutoAI RAG and Milvus database to work with `ibm-watsonx-ai` SDK documentation.

#### Disclaimers

- Use only Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate the usage of IBM AutoAI RAG. The AutoAI RAG experiment conducted in this notebook uses data scraped from the `ibm-watsonx-ai` SDK documentation.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The learning goals of this notebook are:

- Create an AutoAI RAG job that will find the best RAG pattern based on provided data


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [RAG Optimizer definition](#definition)
- [RAG Experiment run](#run)
- [RAG Patterns comparison and testing](#comparison)
- [Historical runs](#runs)
- [Clean up](#cleanup)
- [Summary and next steps](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered; information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the required modules and dependencies

In [ ]:
!pip install -U wget | tail -n 1
!pip install -U 'ibm-watsonx-ai[rag]>=1.3.26' | tail -n 1

### Defining the watsonx.ai credentials
This cell defines the credentials required to work with the watsonx.ai Runtime service.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [2]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your watsonx.ai api key (hit enter): "),
)

### Working with spaces

You need to create a space that will be used for your work. If you do not have a space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=wx) to create one.

- Click **New Deployment Space**
- Create an empty space
- Select Cloud Object Storage
- Select watsonx.ai Runtime instance and press **Create**
- Go to **Manage** tab
- Copy `Space GUID` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: assign space ID below

In [3]:
space_id = 'PASTE YOUR SPACE GUID HERE'

Create an instance of APIClient with authentication details.

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials=credentials, space_id=space_id)

<a id="definition"></a>

## RAG Optimizer definition

### Training data connection

Define connection information to access the COS bucket and the file that contains training data. This example uses [`ibm_watsonx_ai`](https://ibm.github.io/watsonx-ai-python-sdk/index.html) SDK documentation content.

This cell downloads the `ibm_watsonx_ai` Python SDK zip file from GitHub (if not already present), and extracts its contents to a specified folder.

In [7]:
import wget, zipfile, os

filename = "watsonx-ai-python-sdk"
filename_zip = "watsonx-ai-python-sdk.zip"

if not os.path.isfile(filename_zip):
    wget.download("https://github.com/IBM/watsonx-ai-python-sdk/archive/refs/heads/gh-pages.zip", out=filename_zip)

with zipfile.ZipFile(filename_zip, "r") as zip_ref:
    zip_ref.extractall(filename)

In [8]:
from ibm_watsonx_ai.helpers import DataConnection, ContainerLocation

container_location_path = "ibm_watsonx_ai_sdk_docs"

container_data_connection = DataConnection(ContainerLocation(path=f"{container_location_path}/"))
container_data_connection.set_client(client)

Documents with the `.html` extension are filtered and saved to `ContainerLocation`

In [9]:
html_docs_files = []
for root, dirs, files in os.walk(filename):
    if root == f"{filename}/watsonx-ai-python-sdk-gh-pages":
        for file in files:
            if file.endswith('.html'):
                file_path = os.path.join(root, file)
                html_docs_files.append(file_path)

Writing all SDK documents may take around 3 minutes.

In [10]:
import sys

for ind, html_docs_file in enumerate(html_docs_files):
    container_data_connection.write(html_docs_file, remote_name = html_docs_file.split("/")[-1])

    sys.stdout.write(f"\rProgress: {'✓' * (ind+1)}{'.' * (len(html_docs_files)-ind-1)}\r")
    sys.stdout.flush()

### Create a connection to COS
You can skip this section if you already have a connection asset that connects to **IBM Cloud Object Storage**.

In [11]:
datasource_name = 'bluemixcloudobjectstorage'
bucketname = container_data_connection.location.bucket

In [12]:
cos_credentials = client.spaces.get_details(space_id=space_id)['entity']['storage']['properties']

In [13]:
conn_meta_props= {
    client.connections.ConfigurationMetaNames.NAME: f"Connection to Database - {datasource_name} ",
    client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: client.connections.get_datasource_type_id_by_name(datasource_name),
    client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection to external Database",
    client.connections.ConfigurationMetaNames.PROPERTIES: {
        'bucket': bucketname,
        'access_key': cos_credentials['credentials']['editor']['access_key_id'],
        'secret_key': cos_credentials['credentials']['editor']['secret_access_key'],
        'iam_url': 'https://iam.cloud.ibm.com/identity/token',
        'url': cos_credentials['endpoint_url']
    }
}

conn_details = client.connections.create(meta_props=conn_meta_props)
connection_id = client.connections.get_id(conn_details)

Creating connections...
SUCCESS


Finally, we can create a Data Connection that represents input data references.

In [14]:
from ibm_watsonx_ai.helpers import DataConnection, S3Location

input_data_references = [DataConnection(
    connection_asset_id=connection_id,
    location=S3Location(bucket=bucketname,
                        path=container_location_path))]
input_data_references[0].set_client(client)

### Defining a connection to test data

Upload a `json` file that will be used for benchmarking to COS and then define a connection to this file. This example uses content from the [`ibm_watsonx_ai`](https://ibm.github.io/watsonx-ai-python-sdk/index.html) SDK documentation.

In [15]:
benchmarking_data_IBM_page_content = [
    {
        "question": "How to install ibm-watsonx-ai library?",
        "correct_answer": "pip install ibm-watsonx-ai",
        "correct_answer_document_ids": [
            "install.html"
        ]
    },
    {
        "question": "What is Credentails class parameters?",
        "correct_answer": "url, api_key, name, iam_serviceid_crn, token, projects_token, username, password, instance_id, version, bedrock_url, proxies, verify",
        "correct_answer_document_ids": [
            "base.html"
        ]
    },
    {
        "question": "How to get AutoAI pipeline with number 3?",
        "correct_answer": "get_pipeline(pipeline_name='Pipeline_3')",
        "correct_answer_document_ids": [
            "autoai_working_with_class_and_optimizer.html"
        ]
    },
    {
        "question": "How to get list of Embedding Models?",
        "correct_answer": "client.foundation_models.EmbeddingModels",
        "correct_answer_document_ids": [
            "fm_embeddings.html"
        ]
    },
    {
        "question": "How to retrieve the list of model lifecycle data?",
        "correct_answer": "get_model_lifecycle(url='https://us-south.ml.cloud.ibm.com', model_id='ibm/granite-13b-instruct-v2')",
        "correct_answer_document_ids": [
            "fm_helpers.html"
        ]
    },
    {
        "question": "What is path to ModelInference class?",
        "correct_answer": "ibm_watsonx_ai.foundation_models.inference.ModelInference",
        "correct_answer_document_ids": [
            "fm_model_inference.html"
        ]
    },
    {
        "question": "What is method for get model inferance details?",
        "correct_answer": "get_details()",
        "correct_answer_document_ids": [
            "fm_model_inference.html"
        ]
    }
]

The code in the next cell uploads testing data to the bucket as a `json` file.

In [16]:
import json

test_filename = "benchmarking_data_ibm_watson_ai.json"

if not os.path.isfile(test_filename):
    with open(test_filename, "w") as json_file:
        json.dump(benchmarking_data_IBM_page_content, json_file, indent=4)

test_asset_details = client.data_assets.create(name=test_filename, file_path=test_filename)

test_asset_id = client.data_assets.get_id(test_asset_details)
test_asset_id

Creating data asset...
SUCCESS


'9d67c83f-3862-4885-a8a7-58ebf047ebe3'

Define connection information to testing data.

In [18]:
from ibm_watsonx_ai.helpers import DataConnection

test_data_references = [DataConnection(data_asset_id=test_asset_id)]

### Set up connectivity information to Milvus

<b>This notebook focuses on a self-managed Milvus cluster using <a href="https://cloud.ibm.com/docs/watsonxdata?topic=watsonxdata-adding-milvus-service" target="_blank" rel="noopener no referrer">IBM watsonx.data.</a></b>

The following cell retrieves the Milvus username, password, host, and port from the environment (if available) and prompts you to provide them manually in case of failure.

You can provide a connection asset ID to read all required connection data from it. Before doing so, make sure that a connection asset was created in your space.

In [19]:
import os
import getpass

milvus_connection_id = input("Provide connection asset ID in your space. Skip this, if you wish to type credentials by hand and hit enter: ") or None

if milvus_connection_id is None:
    try:
        username = os.environ["USERNAME"]
    except KeyError:
        username = input("Please enter your Milvus user name and hit enter: ")
    try:
        password = os.environ["PASSWORD"]
    except KeyError:
        password = getpass.getpass("Please enter your Milvus password and hit enter: ")
    try:
        host = os.environ["HOST"]
    except KeyError:
        host = input("Please enter your Milvus hostname and hit enter: ")
    try:
        port = os.environ["PORT"]
    except KeyError:
        port = input("Please enter your Milvus port number and hit enter: ")
    try:
        ssl = os.environ["SSL"]
    except:
        ssl = bool(input("Please enter ('y'/anything) if your Milvus instance has SSL enabled. Skip if it is not: "))

    # Create connection
    milvus_data_source_type_id = client.connections.get_datasource_type_uid_by_name(
        "milvus"
    )
    details = client.connections.create(
        {
            client.connections.ConfigurationMetaNames.NAME: "Milvus Connection",
            client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection created by the sample notebook",
            client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: milvus_data_source_type_id,
            client.connections.ConfigurationMetaNames.PROPERTIES: {
                "host": host,
                "port": port,
                "username": username,
                "password": password,
                "ssl": ssl,
            },
        }
    )

    milvus_connection_id = client.connections.get_id(details)

Define connection information to vector store references.

In [20]:
vector_store_references = [DataConnection(connection_asset_id=milvus_connection_id)]

### RAG Optimizer configuration

Provide the input information for AutoAI RAG optimizer:
- `name` - experiment name
- `description` - experiment description
- `max_number_of_rag_patterns` - maximum number of RAG patterns to create
- `optimization_metrics` - target optimization metrics

In [21]:
from ibm_watsonx_ai.experiment import AutoAI

experiment = AutoAI(credentials, space_id=space_id)

rag_optimizer = experiment.rag_optimizer(
    name='AutoAI RAG ibm-watsonx-ai SDK documentation',
    description="AutoAI RAG experiemnt trainded on ibm-watsonx-ai SDK documentataion",
    max_number_of_rag_patterns=6,
    optimization_metrics=[AutoAI.RAGMetrics.ANSWER_CORRECTNESS]
)

Configuration parameters can be retrieved via `get_params()`.

In [22]:
rag_optimizer.get_params()

{'name': 'AutoAI RAG ibm-watsonx-ai SDK documentation',
 'description': 'AutoAI RAG experiemnt trainded on ibm-watsonx-ai SDK documentataion',
 'max_number_of_rag_patterns': 6,
 'optimization_metrics': ['answer_correctness']}

<a id="run"></a>
## RAG Experiment run

Call the `run()` method to trigger the AutoAI RAG experiment. You can either use interactive mode (synchronous job) or background mode (asynchronous job) by specifying `background_mode=True`.

In [23]:
run_details = rag_optimizer.run(
    input_data_references=input_data_references,
    test_data_references=test_data_references,
    vector_store_references=vector_store_references,
    background_mode=False
)



##############################################

Running '09437777-a9e4-4cc6-aac0-96c2c8352d10'

##############################################


pending.............
running......................................................................................................................................................................................................
completed
Training of '09437777-a9e4-4cc6-aac0-96c2c8352d10' finished successfully.


You can use the `get_run_status()` method to monitor AutoAI RAG jobs in background mode.

In [24]:
rag_optimizer.get_run_status()

'completed'

<a id="comparison"></a>
## Comparison and testing of RAG Patterns

You can list the trained patterns and information on evaluation metrics in the form of a Pandas DataFrame by calling the `summary()` method. You can use the DataFrame to compare all discovered patterns and select the one you like for further testing.

In [25]:
summary = rag_optimizer.summary()
summary

,mean_answer_correctness,mean_faithfulness,mean_context_correctness,chunking.method,chunking.chunk_size,chunking.chunk_overlap,embeddings.model_id,vector_store.distance_metric,retrieval.method,retrieval.number_of_chunks,retrieval.hybrid_ranker,generation.model_id
Pattern_Name,,,,,,,,,,,,
Pattern3,0.7857,0.7706,0.8571,recursive,1024,256,ibm/granite-embedding-278m-multilingual,cosine,window,3,"{'k': 60, 'sparse_vectors': {'model_id': 'bm25...",mistralai/mistral-large
Pattern1,0.7418,0.4968,0.6905,recursive,512,128,ibm/granite-embedding-278m-multilingual,cosine,window,3,"{'alpha': 0.8, 'sparse_vectors': {'model_id': ...",ibm/granite-3-8b-instruct
Pattern6,0.6429,0.7091,0.7857,recursive,1024,256,ibm/granite-embedding-278m-multilingual,cosine,window,3,"{'alpha': 0.8, 'sparse_vectors': {'model_id': ...",mistralai/mistral-large
Pattern4,0.5934,0.5517,0.7143,recursive,1024,512,ibm/granite-embedding-278m-multilingual,cosine,window,3,"{'alpha': 0.8, 'sparse_vectors': {'model_id': ...",ibm/granite-3-8b-instruct
Pattern2,0.5000,0.6354,0.7857,recursive,1024,256,ibm/slate-125m-english-rtrvr,cosine,window,3,"{'k': 60, 'sparse_vectors': {'model_id': 'bm25...",ibm/granite-3-8b-instruct
Pattern5,0.5000,0.5715,0.6429,recursive,512,128,ibm/granite-embedding-278m-multilingual,cosine,window,3,"{'k': 60, 'sparse_vectors': {'model_id': 'bm25...",mistralai/mistral-large


Additionally, you can pass the `scoring` parameter to the summary method, to filter RAG patterns starting with the best.

```python
summary = rag_optimizer.summary(scoring="faithfulness")
```

### Get selected pattern

Get the RAGPattern object from the RAG Optimizer experiment. By default, the RAGPattern of the best pattern is returned.

In [26]:
best_pattern_name = summary.index.values[0]
print('Best pattern is:', best_pattern_name)

best_pattern = rag_optimizer.get_pattern()

Best pattern is: Pattern3


The pattern details can be retrieved by calling the `get_pattern_details` method:

```python
rag_optimizer.get_pattern_details(pattern_name='Pattern2')
```

Query the RAGPattern locally, to test it.

In [27]:
from ibm_watsonx_ai.deployments import RuntimeContext

runtime_context = RuntimeContext(api_client=client)
inference_service_function = best_pattern.inference_service(runtime_context)[0]

In [ ]:
question = "How to use new approach of providing credentials to APIClient?"


context = RuntimeContext(
    api_client=client,
    request_payload_json={"messages": [{"role": "user", "content": question}]},
)


resp = inference_service_function(context)

In [29]:
print(inference_service_function(context)["body"]["choices"][0]["message"]["content"])

 </s>To use the new approach of providing credentials to `APIClient`, you should use the `Credentials` class instead of a dictionary. Here's how you can do it:

1. Import the necessary classes from `ibm_watsonx_ai`.
2. Create an instance of the `Credentials` class with the required parameters.
3. Pass the `Credentials` instance to the `APIClient` constructor.

Here's an example:

```python
from ibm_watsonx_ai import APIClient, Credentials

# Create a Credentials instance
credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    token="***********"
)

# Initialize the APIClient with the Credentials instance
client = APIClient(credentials)

# Now you can use the client to interact with the API
url = client.credentials.url
# OR
url = credentials.url
```

This approach ensures that you are using the latest and recommended method for providing credentials to the `APIClient`.


### Deploy RAGPattern

Deployment is done by storing the defined RAG function and then by creating a deployed asset.

In [30]:
deployment_details = best_pattern.inference_service.deploy(
    name="AutoAI RAG deployment - ibm_watsonx_ai documentataion",
    space_id=space_id,
    deploy_params={"tags": ["wx-autoai-rag"]}
)



######################################################################################

Synchronous deployment creation for id: 'f54af56a-9d55-4b58-8db1-5b3cd1ed2673' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
..........
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='cfabea10-76b5-4a8c-ae43-b3e4c9d70885'
-----------------------------------------------------------------------------------------------




### Test the deployed function

RAG service is now deployed in our space. To test our solution we can run the cell below. Questions have to be provided in the payload. Their format is provided below.

In [31]:
deployment_id = client.deployments.get_id(deployment_details)

payload = {
    "messages": [{"role": "user", "content": question}]
}
score_response = client.deployments.run_ai_service(deployment_id, payload)

In [32]:
score_response

{'choices': [{'index': 0,
   'message': {'content': ' </s>To use the new approach of providing credentials to `APIClient`, you should use the `Credentials` class instead of a dictionary. Here\'s how you can do it:\n\n1. Import the necessary classes from `ibm_watsonx_ai`.\n2. Create an instance of the `Credentials` class with the required parameters.\n3. Pass the `Credentials` instance to the `APIClient` constructor.\n\nHere\'s an example:\n\n```python\nfrom ibm_watsonx_ai import APIClient, Credentials\n\n# Create a Credentials instance\ncredentials = Credentials(\n    url="https://us-south.ml.cloud.ibm.com",\n    token="***********"\n)\n\n# Initialize the APIClient with the Credentials instance\nclient = APIClient(credentials)\n\n# Now you can use the client to interact with the API\nurl = client.credentials.url\n# OR\nurl = credentials.url\n```\n\nThis approach ensures that you are using the latest and recommended method for providing credentials to the `APIClient`.',
    'role': 'ass

In [33]:
print(score_response["choices"][0]["message"]["content"])

 </s>To use the new approach of providing credentials to `APIClient`, you should use the `Credentials` class instead of a dictionary. Here's how you can do it:

1. Import the necessary classes from `ibm_watsonx_ai`.
2. Create an instance of the `Credentials` class with the required parameters.
3. Pass the `Credentials` instance to the `APIClient` constructor.

Here's an example:

```python
from ibm_watsonx_ai import APIClient, Credentials

# Create a Credentials instance
credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    token="***********"
)

# Initialize the APIClient with the Credentials instance
client = APIClient(credentials)

# Now you can use the client to interact with the API
url = client.credentials.url
# OR
url = credentials.url
```

This approach ensures that you are using the latest and recommended method for providing credentials to the `APIClient`.


<a id="runs"></a>
## Historical runs

In this section you learn to work with historical RAG Optimizer jobs (runs).

To list historical runs use the `list()` method and provide the `'rag_optimizer'` filter.

In [34]:
experiment.runs(filter='rag_optimizer').list()

,timestamp,run_id,state,auto_pipeline_optimizer name
0,2025-06-25T13:42:45.199Z,09437777-a9e4-4cc6-aac0-96c2c8352d10,completed,AutoAI RAG ibm-watsonx-ai SDK documentation


In [35]:
run_id = run_details['metadata']['id']
run_id

'09437777-a9e4-4cc6-aac0-96c2c8352d10'

### Get executed optimizer's configuration parameters

In [37]:
experiment.runs.get_rag_params(run_id=run_id)

{'name': 'AutoAI RAG ibm-watsonx-ai SDK documentation',
 'description': 'AutoAI RAG experiemnt trainded on ibm-watsonx-ai SDK documentataion',
 'max_number_of_rag_patterns': 6,
 'optimization_metrics': ['answer_correctness']}

### Get historical rag_optimizer instance and training details

In [38]:
historical_opt = experiment.runs.get_rag_optimizer(run_id)

### List trained patterns for selected optimizer

In [39]:
historical_opt.summary()

,mean_answer_correctness,mean_faithfulness,mean_context_correctness,chunking.method,chunking.chunk_size,chunking.chunk_overlap,embeddings.model_id,vector_store.distance_metric,retrieval.method,retrieval.number_of_chunks,retrieval.hybrid_ranker,generation.model_id
Pattern_Name,,,,,,,,,,,,
Pattern3,0.7857,0.7706,0.8571,recursive,1024,256,ibm/granite-embedding-278m-multilingual,cosine,window,3,"{'k': 60, 'sparse_vectors': {'model_id': 'bm25...",mistralai/mistral-large
Pattern1,0.7418,0.4968,0.6905,recursive,512,128,ibm/granite-embedding-278m-multilingual,cosine,window,3,"{'alpha': 0.8, 'sparse_vectors': {'model_id': ...",ibm/granite-3-8b-instruct
Pattern6,0.6429,0.7091,0.7857,recursive,1024,256,ibm/granite-embedding-278m-multilingual,cosine,window,3,"{'alpha': 0.8, 'sparse_vectors': {'model_id': ...",mistralai/mistral-large
Pattern4,0.5934,0.5517,0.7143,recursive,1024,512,ibm/granite-embedding-278m-multilingual,cosine,window,3,"{'alpha': 0.8, 'sparse_vectors': {'model_id': ...",ibm/granite-3-8b-instruct
Pattern2,0.5000,0.6354,0.7857,recursive,1024,256,ibm/slate-125m-english-rtrvr,cosine,window,3,"{'k': 60, 'sparse_vectors': {'model_id': 'bm25...",ibm/granite-3-8b-instruct
Pattern5,0.5000,0.5715,0.6429,recursive,512,128,ibm/granite-embedding-278m-multilingual,cosine,window,3,"{'k': 60, 'sparse_vectors': {'model_id': 'bm25...",mistralai/mistral-large


<a id="cleanup"></a>
## Clean up

To delete the current experiment, use the `cancel_run` method.

**Warning:** Be careful: once you delete an experiment, you will no longer be able to refer to it.

In [40]:
rag_optimizer.cancel_run()

'SUCCESS'

To delete the deployment, use the `delete` method. 

**Warning:** Keeping the deployment active may lead to unnecessary consumption of Compute Unit Hours (CUHs).

In [41]:
client.deployments.delete(deployment_id)

'SUCCESS'

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to use `ibm-watsonx-ai` to run AutoAI RAG experiments. 

 Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Michał Steczko**, Software Engineer at watsonx.ai

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.